In [1]:
import imageio
import math
from numpy import uint8

In [2]:
#ToDo...This checks if the file selected is an image or not
#def checkIfImg(imgLoc):

In [3]:
def extractImgData(imgLoc):
    pixelMatrix = imageio.imread(imgLoc) #use imsave(filename, pixelMatrix) to save the image
    return pixelMatrix

In [4]:
def grayscale(height, width, pixelMatrix):
    for i in range(height):
        for j in range(width):
            avgCol = ((int)(pixelMatrix[i,j,0]) + (int)(pixelMatrix[i,j,1]) + (int)(pixelMatrix[i,j,2]))/3
            pixelMatrix[i,j,0] = pixelMatrix[i,j,1] = pixelMatrix[i,j,2] = uint8(avgCol)

In [5]:
def vertical_flip(height, width, pixelMatrix):
    i = 0
    while (i < height):
        j = 0
        while (j < width/2):
            redTemp = pixelMatrix[i,j,0]
            pixelMatrix[i,j,0] = pixelMatrix[i,width-1-j,0]
            pixelMatrix[i,width-1-j,0] = redTemp
            
            greenTemp = pixelMatrix[i,j,1]
            pixelMatrix[i,j,1] = pixelMatrix[i,width-1-j,1]
            pixelMatrix[i,width-1-j,1] = greenTemp
            
            blueTemp = pixelMatrix[i,j,2]
            pixelMatrix[i,j,2] = pixelMatrix[i,width-1-j,2]
            pixelMatrix[i,width-1-j,2] = blueTemp
            
            j += 1
        i += 1

In [6]:
def horizontal_flip(height,width,pixelMatrix):
    i = 0
    while (i < height/2):
        j = 0
        while (j < width):
            redTemp = pixelMatrix[i,j,0]
            pixelMatrix[i,j,0] = pixelMatrix[height-1-i,j,0]
            pixelMatrix[height-1-i,j,0] = redTemp
            
            greenTemp = pixelMatrix[i,j,1]
            pixelMatrix[i,j,1] = pixelMatrix[height-1-i,j,1]
            pixelMatrix[height-1-i,j,1] = greenTemp
            
            blueTemp = pixelMatrix[i,j,2]
            pixelMatrix[i,j,2] = pixelMatrix[height-1-i,j,2]
            pixelMatrix[height-1-i,j,2] = blueTemp
            
            j += 1
        i += 1

In [7]:
def init_surr_cells(pixel):
    surr_cells = [[[0,0,0],[0,0,0],[0,0,0]],[[0,0,0],[pixel[0], pixel[1], pixel[2]],[0,0,0]],[[0,0,0],[0,0,0],[0,0,0]]]
    return surr_cells

In [8]:
def Sobel_Operator(surr_cells, color):
    GX = [[-1,0,1],[-2,0,2],[-1,0,1]]
    GY = [[-1,-2,-1],[0,0,0],[1,2,1]]
    val_gx = val_gy = 0
    if color == 'r':
        for i in range(3):
            for j in range(3):
                val_gx = val_gx + surr_cells[i][j][0]*GX[i][j]
                val_gy = val_gy + surr_cells[i][j][0]*GY[i][j]
    elif color == 'g':
        for i in range(3):
            for j in range(3):
                val_gx = val_gx + surr_cells[i][j][1]*GX[i][j]
                val_gy = val_gy + surr_cells[i][j][1]*GY[i][j]
    else:
        for i in range(3):
            for j in range(3):
                val_gx = val_gx + surr_cells[i][j][2]*GX[i][j]
                val_gy = val_gy + surr_cells[i][j][2]*GY[i][j]
    net_val = round(math.sqrt(val_gx**2 + val_gy**2))
    if net_val > 255:
        net_val = 255
    return net_val

In [9]:
def edgify(height,width,pixelMatrix):
    temp = list()
    i = 0
    while i < height:
        j = 0
        while j < width:
            surr_cells = init_surr_cells(pixelMatrix[i,j])
            
            if i != height - 1:
                surr_cells[2][1][0] = pixelMatrix[i+1,j,0]
                surr_cells[2][1][1] = pixelMatrix[i+1,j,1]
                surr_cells[2][1][2] = pixelMatrix[i+1,j,2]
                if j != width - 1:
                    surr_cells[2][2][0] = pixelMatrix[i+1,j+1,0]
                    surr_cells[2][2][1] = pixelMatrix[i+1,j+1,1]
                    surr_cells[2][2][2] = pixelMatrix[i+1,j+1,2]
                if j != 0:
                    surr_cells[2][0][0] = pixelMatrix[i+1,j-1,0]
                    surr_cells[2][0][1] = pixelMatrix[i+1,j-1,1]
                    surr_cells[2][0][2] = pixelMatrix[i+1,j-1,2]
                 
            if i != 0:
                surr_cells[0][1][0] = pixelMatrix[i-1,j,0]
                surr_cells[0][1][1] = pixelMatrix[i-1,j,1]
                surr_cells[0][1][2] = pixelMatrix[i-1,j,2]
                if j != width - 1:
                    surr_cells[0][2][0] = pixelMatrix[i-1,j+1,0]
                    surr_cells[0][2][1] = pixelMatrix[i-1,j+1,1]
                    surr_cells[0][2][2] = pixelMatrix[i-1,j+1,2]
                if j != 0:
                    surr_cells[0][0][0] = pixelMatrix[i-1,j-1,0]
                    surr_cells[0][0][1] = pixelMatrix[i-1,j-1,1]
                    surr_cells[0][0][2] = pixelMatrix[i-1,j-1,2]
                 
            if j != width - 1:
                surr_cells[1][2][0] = pixelMatrix[i,j+1,0]
                surr_cells[1][2][1] = pixelMatrix[i,j+1,1]
                surr_cells[1][2][2] = pixelMatrix[i,j+1,2]
             
            if j != 0:
                surr_cells[1][0][0] = pixelMatrix[i,j-1,0]
                surr_cells[1][0][1] = pixelMatrix[i,j-1,1]
                surr_cells[1][0][2] = pixelMatrix[i,j-1,2]
            
            if i == 1 and j == 1:
                print(pixelMatrix[i,j])
                print(surr_cells)
                #print(surr_cells[1][1][0] + surr_cells[1][0][0])
            
            x = Sobel_Operator(surr_cells, 'r') 
            y = Sobel_Operator(surr_cells, 'g') 
            z = Sobel_Operator(surr_cells, 'b') 
            temp.append([x,y,z])
            j += 1
        print((int)(i*100/height),"%")
        i += 1
    i = 0
    k = 0
    while i < height:
        j = 0
        while j < width:
            pixelMatrix[i,j,0] = (uint8)(temp[k][0])
            pixelMatrix[i,j,1] = (uint8)(temp[k][1])
            pixelMatrix[i,j,2] = (uint8)(temp[k][2])
            k += 1
            j += 1
        i += 1
    print(pixelMatrix[1,1])

In [10]:
def solarization(height, width, pixelMatrix, threshold):
    i = 0
    while i < height:
        j = 0
        while j < width:
            if pixelMatrix[i,j,0] >= threshold:
                pixelMatrix[i,j,0] = 255 - pixelMatrix[i,j,0]
                
            if pixelMatrix[i,j,1] >= threshold:
                pixelMatrix[i,j,1] = 255 - pixelMatrix[i,j,1]
                
            if pixelMatrix[i,j,2] >= threshold:
                pixelMatrix[i,j,2] = 255 - pixelMatrix[i,j,2]
            j += 1
        i += 1

In [11]:
def edgify1(height,width,pixelMatrix):
    i = j = 0
    surr_cells = init_surr_cells(pixelMatrix[i,j])
            
    if i != height - 1:
        surr_cells[2][1] = pixelMatrix[i+1,j]
        if j != width - 1:
            surr_cells[2][2] = pixelMatrix[i+1,j+1]
        if j != 0:
            surr_cells[2][0] = pixelMatrix[i+1,j-1] 

    if i != 0:
        surr_cells[0][1] = pixelMatrix[i-1,j]
        if j != width - 1:
            surr_cells[0][2] = pixelMatrix[i-1,j+1]
        if j != 0:
            surr_cells[0][0] = pixelMatrix[i-1,j-1]

    if j != width - 1:
        surr_cells[1][2] = pixelMatrix[i,j+1]

    if j != 0:
        surr_cells[1][0] = pixelMatrix[i,j-1]
    '''
    if i == 0 and j == 0:
        print(surr_cells)
        print(surr_cells[1][1][0] + surr_cells[1][0][0])
    '''
    pixelMatrix[i,j,0] = Sobel_Operator(surr_cells, 'r') 
    pixelMatrix[i,j,1] = Sobel_Operator(surr_cells, 'g') 
    pixelMatrix[i,j,2] = Sobel_Operator(surr_cells, 'b')
    print(pixelMatrix[i,j])

In [12]:
def brightness(height, width, pixelMatrix,scalingFactor):
        i=0;
        j=0;
        while(i<height):
            j=0
            while(j<width):
                
                
                if(pixelMatrix[i,j,0]+scalingFactor>255):
                    pixelMatrix[i,j,0]=255
                else:
                     pixelMatrix[i,j,0]=pixelMatrix[i,j,0]+scalingFactor
                
                if(pixelMatrix[i,j,1]+scalingFactor>255):
                    pixelMatrix[i,j,1]=255
                else:
                    pixelMatrix[i,j,1]=pixelMatrix[i,j,1]+scalingFactor
                
                
                    
                if(pixelMatrix[i,j,2]+scalingFactor>255):
                    pixelMatrix[i,j,2]=255
                else:
                    pixelMatrix[i,j,2]=pixelMatrix[i,j,2]+scalingFactor
                
                j+=1
            i+=1

In [13]:
   def contrast(height,width, pixelMatrix, contrastFactor):
        i=0
        j=0
        F=(259*(255+contrastFactor))/(255*(259-contrastFactor))
        
        while(i<height):
            j=0
            while(j<width):
                if((F*(pixelMatrix[i,j,0]-128)) + 128 >255):
                    pixelMatrix[i,j,0]=255
                elif((F*(pixelMatrix[i,j,0]-128)) + 128 <0):
                    pixelMatrix[i,j,0]=0
                else:
                    pixelMatrix[i,j,0]=(F*(pixelMatrix[i,j,0]-128)) + 128 
                if((F*(pixelMatrix[i,j,1]-128)) + 128 >255):
                    pixelMatrix[i,j,1]=255
                elif((F*(pixelMatrix[i,j,1]-128)) + 128 <0):
                    pixelMatrix[i,j,1]=0
                else:
                    pixelMatrix[i,j,1]=(F*(pixelMatrix[i,j,1]-128)) + 128
                if((F*(pixelMatrix[i,j,2]-128)) + 128 > 255):
                    pixelMatrix[i,j,2]=255
                elif((F*(pixelMatrix[i,j,2]-128)) + 128 <0):
                    pixelMatrix[i,j,2]=0
                else:
                    pixelMatrix[i,j,2]=(F*(pixelMatrix[i,j,2]-128)) + 128
                j+=1
            i+=1
            

In [14]:
    def blur(height, width, pixelMatrix):
        temp=list()
        avgR=0
        avgG=0
        avgB=0
        i=0
        j=0
        while(i<height):
            j=0
            while(j<width):
                avgR=0.0
                avgG=0.0
                avgR=0.0
                neighbors=list()
                #1st of 3*3 Matrix
                neighbors.append([pixelMatrix[i,j,0],pixelMatrix[i,j,1],pixelMatrix[i,j,2]])
                totalNeighbors=1
                
                if(i!=height-1):
                                 neighbors.append([pixelMatrix[i+1,j,0],pixelMatrix[i+1,j,1],pixelMatrix[i+1,j,2]])
                                 totalNeighbors+=1
                                                   
                                 if(j!=width-1):
                                                   neighbors.append([pixelMatrix[i+1,j+1,0],pixelMatrix[i+1,j+1,1],pixelMatrix[i+1,j+1,2]])
                                                   totalNeighbors+=1
                                
                                 if(j!=0):
                                                   neighbors.append([pixelMatrix[i+1,j-1,0],pixelMatrix[i+1,j-1,1],pixelMatrix[i+1,j-1,2]])
                                                   totalNeighbors+=1
                                                
                if(i!=0):
                                neighbors.append([pixelMatrix[i-1,j,0],pixelMatrix[i-1,j,1],pixelMatrix[i-1,j,2]])
                                totalNeighbors+=1
                            
                                if(j!=width-1):
                                                  neighbors.append([pixelMatrix[i-1,j+1,0],pixelMatrix[i-1,j+1,1],pixelMatrix[i-1,j+1,2]])
                                                  totalNeighbors+=1
                                            
                                if(j!=0):
                                                  neighbors.append([pixelMatrix[i-1,j-1,0],pixelMatrix[i-1,j-1,1],pixelMatrix[i-1,j-1,2]])
                                                  totalNeighbors+=1
                                            
                if(j!=width-1):
                                neighbors.append([pixelMatrix[i,j+1,0],pixelMatrix[i,j+1,1],pixelMatrix[i,j+1,2]])
                                totalNeighbors+=1
                            
                if(j!=0):
                                neighbors.append([pixelMatrix[i,j-1,0],pixelMatrix[i,j-1,1],pixelMatrix[i,j-1,2]])
                                totalNeighbors+=1
                k=0
                                                   
                for x in range(len(neighbors)):
                                                  avgR+=neighbors[k][0]
                                                  avgG+=neighbors[k][1]
                                                  avgB+=neighbors[k][2]
                                                  k=k+1
                
                avgR=avgR/totalNeighbors
                avgB=avgB/totalNeighbors
                avgG=avgG/totalNeighbors
            
                temp.append([avgR,avgG,avgB])
                j=j+1
                
        i=i+1
        
        f=0
        g=0
        l=0
        while(f<height):
                            g=0
                            while(g<width):
                                                      
                                                      pixelMatrix[f][g][0]=(uint8)(temp[l][0])
                                                      pixelMatrix[f][g][1]=(uint8)(temp[l][1])
                                                      pixelMatrix[f][g][2]=(uint8)(temp[l][2])
                                                      
                                                      l=l+1
                                                      g=g+1
        f=f+1
                                       
            
         

In [15]:
#Choose Image and extract pixels matrix for the image
pixelMatrix = extractImgData("nature.jpeg")
height = len(pixelMatrix)
width = len(pixelMatrix[0])
#print(pixelMatrix[0,0,0]*2,pixelMatrix[0,0,0]*-2)
#Edits on the image
#Done...grayscale(height,width,pixelMatrix)
#Done...vertical_flip(height, width, pixelMatrix)
#Done...horizontal_flip(height, width, pixelMatrix)
#Done...edgify(height,width,pixelMatrix)
#Done...solarization(height,width,pixelMatrix,0)
# brightness(height, width, pixelMatrix, 100)
# contrast(height,width, pixelMatrix, 40)
#not saving the file(Error)....blur(height, width, pixelMatrix)
#Saving the image
imageio.imsave("nature-edited.jpeg", pixelMatrix);